In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("Plots")
Pkg.add("Combinatorics")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed StructTypes ──────── v1.11.0
   Installed CodecBzip2 ───────── v0.8.5
   Installed JSON3 ────────────── v1.14.3
   Installed BenchmarkTools ───── v1.6.3
   Installed MutableArithmetics ─ v1.6.7
   Installed MathOptInterface ─── v1.48.0
   Installed JuMP ─────────────── v1.29.3
    Updating `~/.julia/environments/v1.11/Project.toml`
  [4076af6c] + JuMP v1.29.3
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [523fee87] + CodecBzip2 v0.8.5
  [0f8b85d8] + JSON3 v1.14.3
  [4076af6c] + JuMP v1.29.3
  [b8f27783] + MathOptInterface v1.48.0
  [d8a4904e] + MutableArithmetics v1.6.7
  [856f2bd8] + StructTypes v1.11.0
  [9abbd945] + Profile v1.11.0
Precompiling project...
   4708.2 ms  ✓ CodecBzip2
   5185.1 ms  ✓ StructTypes
   3406.9 ms  ✓ BenchmarkTools
  17255.3 ms  ✓ MutableArithmetics
  19708.6 ms  ✓ JSON3
  82762.8 ms  ✓ MathOptInterface
 

In [1]:
using JuMP, HiGHS
using Plots
using LinearAlgebra
using Random
using Combinatorics

#MODELO JUGUETE

Solucion ={capacidad del vehiculo, ruta}

{50,{1,3,4,7}}

{18,{2}}

{9,{5}}

{5,{6}}

In [2]:
#TOY MODEL
n_camiones = 4
n_clientes = 7
capacidades = [50,18,9,5] #solo 4 camiones
demandas_clientes=[10,15,10,10,7,4,20] #solo 7 clientes
c_clientes = collect(1:n_clientes) #Conjunto de clientes
PJ = [collect(c) for k in 1:length(c_clientes) for c in combinations(c_clientes, k)] #Conjunto potencia de clientes
card_PJ = length(PJ) #Número de rutas segun ruta de clientes
dj = zeros(card_PJ) #valores dj del problema

for i in 1:card_PJ
  sum=0
  for j in PJ[i]
    sum+=demandas_clientes[j]
  end
  dj[i] = sum
end

rj = ones(card_PJ)*10000

#Simulamos la funcion indicadora
S_R = zeros(n_camiones,card_PJ)
for i in 1:n_camiones
  for j in 1:card_PJ
    S_R[i,j] = capacidades[i] + rj[j]
  end
end

M = Model(HiGHS.Optimizer)
@variable(M, x[1:n_camiones, 1:card_PJ] >= 0, Bin)
@objective(M, Min, sum(x[i,j] * S_R[i,j] for i in 1:n_camiones, j in 1:card_PJ))
@constraint(M, [i in 1:n_camiones], sum(x[i,j] for j in 1:card_PJ) <= 1)
@constraint(M, [i in 1:n_camiones], sum(x[i,j]*dj[j] for j in 1:card_PJ) <= capacidades[i])
@constraint(M, [k in 1:n_clientes], sum(x[i,j]*Int(k in PJ[j]) for i in 1:n_camiones,j in 1:card_PJ) == 1)
optimize!(M)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 15 rows; 508 cols; 2808 nonzeros; 508 integer variables (508 binary)
Coefficient ranges:
  Matrix  [1e+00, 8e+01]
  Cost    [1e+04, 1e+04]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 5e+01]
Presolving model
14 rows, 113 cols, 543 nonzeros  0s
11 rows, 52 cols, 196 nonzeros  0s
10 rows, 19 cols, 69 nonzeros  0s
9 rows, 17 cols, 53 nonzeros  0s
Presolve reductions: rows 9(-6); columns 17(-491); nonzeros 53(-2755) 
Objective function is integral with scale 1

Solving MIP model with:
   9 rows
   17 cols (17 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   53 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Triv

Presentando la solucion:

In [3]:
z_opt = objective_value(M)
X_opt = value.(x)

rutas_posiciones = sum(X_opt, dims=1) #Suma de columnas
vehiculos_empleados = round(Int,sum(sum(X_opt, dims =2)))

rutas_solucion = []

for i in 1:card_PJ
  if rutas_posiciones[i]>0.001
    for j in 1:n_camiones
      if X_opt[j,i]>0.00001
        push!(rutas_solucion,[j,PJ[i]])
      end
    end
  end
end

println("Se emplearon ",vehiculos_empleados," vehiculos")
for i in rutas_solucion
  println("El vehiculo ",i[1]," para por los clientes: ",i[2])
end

Se emplearon 4 vehiculos
El vehiculo 2 para por los clientes: [2]
El vehiculo 3 para por los clientes: [5]
El vehiculo 4 para por los clientes: [6]
El vehiculo 1 para por los clientes: [1, 3, 4, 7]


#PROBLEMA CON DATOS GENERADOS ALEATORIAMENTE

In [2]:
##########################################
#Capacidades de los camiones
n_camiones =10 #Numero de camiones
tipos_camiones=[0.65,0.15,0.05,0.15] #Proporciones de camiones
capacidades_camiones = [500,800,1500,2000] #Capacidades por tipo de camiones
##########################################

n_tipo_camion=zeros(Int,length(tipos_camiones))
for i in 1:(length(tipos_camiones)-1)
  n_tipo_camion[i]=round(Int,n_camiones*tipos_camiones[i])
end
n_tipo_camion[length(tipos_camiones)] = n_camiones-sum(n_tipo_camion) #Numero de camiones por tipo

capacidades = reduce(vcat, [fill(capacidades_camiones[i], n_tipo_camion[i]) for i in eachindex(n_tipo_camion)]); #Capacidades de cada camion

##########################################
#Demandas de clientes
Random.seed!(123) #fijar la semilla del generador de números aleatorios
n_clientes = 15 #Numero de clientes
c_clientes = collect(1:n_clientes) #Conjunto de clientes
demandas_clientes = rand(200:600, n_clientes); #Generando demandas de clientes

In [3]:
demandas_sort =  sort(demandas_clientes)
println(demandas_sort)
demandas_acum = cumsum(demandas_sort)
println(demandas_acum)
max_capacity = maximum(capacidades)
n_max_comb = 0
for i in 1:length(demandas_acum)
  if demandas_acum[i]>max_capacity
      n_max_comb = i-1
    break
  end
end
println("Maxima capacidad: ",max_capacity)
println("Numero maximo de combinaciones: ",n_max_comb)
println("Numero de combinacion: ",binomial(n_clientes,n_max_comb))

[216, 217, 276, 331, 345, 356, 386, 409, 410, 411, 432, 435, 535, 557, 574]
[216, 433, 709, 1040, 1385, 1741, 2127, 2536, 2946, 3357, 3789, 4224, 4759, 5316, 5890]
Maxima capacidad: 2000
Numero maximo de combinaciones: 6
Numero de combinacion: 5005


In [4]:
PJ = [collect(c) for k in 1:length(c_clientes) for c in combinations(c_clientes, k)] #Conjunto potencia de clientes
#PJ = [collect(c) for k in 1:n_max_comb for c in combinations(c_clientes, k)] #Conjunto potencia de clientes
card_PJ = length(PJ) #Número de rutas segun ruta de clientes
dj = zeros(card_PJ) #valores dj del problema
for i in 1:card_PJ
  sum=0
  for j in PJ[i]
    sum+=demandas_clientes[j]
  end
  dj[i] = sum
end
rj = rand(500:800, card_PJ); #Vector del costo de cada ruta de clientes


Resolviendo el problema

In [6]:
M = Model(HiGHS.Optimizer)
@variable(M, x[1:n_camiones, 1:card_PJ] >= 0, Bin)

@objective(M, Min, sum(x[i,j] * (capacidades[i] + rj[j]) for i in 1:n_camiones, j in 1:card_PJ))
@constraint(M, [i in 1:n_camiones], sum(x[i,j] for j in 1:card_PJ) <= 1)
@constraint(M, [i in 1:n_camiones], sum(x[i,j]*dj[j] for j in 1:card_PJ) <= capacidades[i])
@constraint(M, [k in 1:n_clientes], sum(x[i,j]*Int(k in PJ[j]) for i in 1:n_camiones,j in 1:card_PJ) == 1)
optimize!(M)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 35 rows; 327670 cols; 3112940 nonzeros; 327670 integer variables (327670 binary)
Coefficient ranges:
  Matrix  [1e+00, 6e+03]
  Cost    [1e+03, 3e+03]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 2e+03]
Presolving model
35 rows, 7994 cols, 50276 nonzeros  6s
35 rows, 7994 cols, 50276 nonzeros  8s
Presolve reductions: rows 35(-0); columns 7994(-319676); nonzeros 50276(-3062664) 
Objective function is integral with scale 1

Solving MIP model with:
   35 rows
   7994 cols (7994 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   50276 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial poin

In [7]:
z_opt = objective_value(M)
X_opt = value.(x)
println("Valor óptimo de la función objetivo = ", z_opt)
println("Solución óptima de X =")
display(X_opt)

Valor óptimo de la función objetivo = 8694.0
Solución óptima de X =


10×32767 Matrix{Float64}:
 1.0  0.0  0.0  0.0   0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  -0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [8]:
rutas_posiciones = sum(X_opt, dims=1) #Suma de columnas
vehiculos_empleados = round(Int,sum(sum(X_opt, dims =2)))

rutas_solucion = []

for i in 1:card_PJ
  if rutas_posiciones[i]>0.001
    for j in 1:n_camiones
      if X_opt[j,i]>0.001
        push!(rutas_solucion,[j,PJ[i]])
      end
    end
  end
end

println("Se emplearon ",vehiculos_empleados," vehiculos")
for i in rutas_solucion
  println("El vehiculo ",i[1]," para por los clientes: ",i[2],"(capacidad: ",capacidades[i[1]],")")
end

Se emplearon 5 vehiculos
El vehiculo 1 para por los clientes: [1](capacidad: 500)
El vehiculo 7 para por los clientes: [11, 15](capacidad: 800)
El vehiculo 8 para por los clientes: [6, 7, 13](capacidad: 800)
El vehiculo 10 para por los clientes: [3, 8, 10, 12](capacidad: 2000)
El vehiculo 9 para por los clientes: [2, 4, 5, 9, 14](capacidad: 2000)


#OTRA SOLUCION:

In [9]:
idx = sortperm(dj) #indices ordenados segun demandas de cada ruta
dj_sort = dj[idx];

In [10]:
R_i = []
for i in 1:n_camiones
rutas =[]
  for j in 1:card_PJ
    if dj_sort[j]>capacidades[i]
      break
    end
    push!(rutas,idx[j])
  end
  push!(R_i, rutas)
end

In [11]:
M = Model(HiGHS.Optimizer)

x = Dict{Tuple{Int, Int}, VariableRef}()

for i in 1:n_camiones
  for j in 1:(length(R_i[i]))

    x[(i,j)] = @variable(M, binary = true, base_name = "x_$(i)_$(j)")
  end
end

@objective(M, Min, sum(x[(i, j)]*(capacidades[i] + rj[R_i[i][j]]) for (i,j) in keys(x)))
for i in 1:n_camiones
  @constraint(M, sum(x[(i, j)] for j in 1:(length(R_i[i]))) <= 1)
end

for k in 1:n_clientes

  @constraint(M, sum(sum(x[(i, j)]*Int(k in PJ[R_i[i][j]]) for j in 1:(length(R_i[i]))) for i in 1:n_camiones) == 1)
end

optimize!(M)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 25 rows; 7994 cols; 42282 nonzeros; 7994 integer variables (7994 binary)
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [1e+03, 3e+03]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 1e+00]
Presolving model
25 rows, 7994 cols, 42282 nonzeros  0s
25 rows, 7994 cols, 42282 nonzeros  1s
Presolve reductions: rows 25(-0); columns 7994(-0); nonzeros 42282(-0) - Not reduced
Objective function is integral with scale 1

Solving MIP model with:
   25 rows
   7994 cols (7994 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   42282 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u =

In [12]:
for (k, v) in x
  if value(v)>0.001
    println("vehiculo: ",k[1],",ruta: ",PJ[R_i[k[1]][k[2]]],", capacidad: ",capacidades[k[1]])
  end
end

vehiculo: 10,ruta: [2, 4, 5, 9, 14], capacidad: 2000
vehiculo: 8,ruta: [6, 7, 13], capacidad: 800
vehiculo: 7,ruta: [11, 15], capacidad: 800
vehiculo: 9,ruta: [3, 8, 10, 12], capacidad: 2000
vehiculo: 4,ruta: [1], capacidad: 500
